In [1]:
import pyspark
import datetime
import re
import string

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
31,application_1638946947785_0033,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
start = datetime.datetime.now()
input_rdd=sc.textFile("s3://millionsongs-data/new_g4g.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
first_line= input_rdd.first()
input_rdd =input_rdd.filter(lambda x: x!=first_line)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import datetime
start = datetime.datetime.now()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
columns=first_line.split("\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def index(col):
    return columns.index(col)
partition_ct=10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
analysis= input_rdd.map(lambda x : x.split("\t")).\
map(lambda x : ( x[index("get_year")] , x[index("get_song_id")],x[index("get_artist_mbtags")],x[index("get_loudness")],x[index("get_duration")]))\
.filter(lambda x :x[0]!='TRCROFJ128F4290DAE' and x[0]!='0' and x[0]!='2009' and x[0]!='2010' and x[0]!='2011')\
.map(lambda x: ( int(x[0]), x[1],x[2],x[3],x[4] )).sortBy(lambda x:x[0],numPartitions=partition_ct)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1. Number of Songs over years

In [8]:
output2= analysis.map(lambda x:(x[0],1))\
.reduceByKey(lambda x,y:x+y,numPartitions=partition_ct)\
.sortBy(lambda x:x[0],numPartitions=partition_ct)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
viz2_df=output2.toDF()
viz2_df.createOrReplaceTempView("viz2_tab")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%sql -o analysis2_df
SELECT * FROM viz2_tab

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [11]:
%%local
import plotly.express as px
fig = px.line(analysis2_df, x='_1', y='_2')
fig.update_layout(
    height=400,
    title_text='  #Number of songs produced over years',
    yaxis_title="#Number of songs",
    xaxis_title="Year"
)
fig.show()

## 2. Number of Genres over year

In [12]:
def clean_mbtags(z):
    a=[]
    z=z.lower()
    z = re.sub('[' + string.punctuation + '0-9\\r\\t\\n]', ' ', z)
    z = [word.strip() for word in z.split("   ")]
    return z

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
cleaned= analysis.map(lambda x:(x[0],x[2]))\
.reduceByKey(lambda x,y:x+y,numPartitions=partition_ct)\
.map(lambda x: ( x[0],clean_mbtags(x[1]))).cache()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#analysis.unpersist()

output1=cleaned.map(lambda x: (x[0],len(set(x[1]))))\
.map(lambda x:(int(x[0]),x[1])).sortBy(lambda x:x[0],numPartitions=partition_ct)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
viz1_df=output1.toDF()
viz1_df.createOrReplaceTempView("viz1_tab")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%sql -o analysis1_df
SELECT * FROM viz1_tab

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [17]:
%%local
import plotly.express as px
fig = px.line(analysis1_df, x='_1', y='_2')
fig.update_layout(
    height=400,
    title_text='  #Genres of Songs Produced over years',
    yaxis_title="#Genres",
    xaxis_title="Year"
)
fig.show()

## 3. Evolution of selected genres over year

In [18]:
def get_trend(x,genre):
    unique,counts=np.unique(x,return_counts=True)
    print(unique)
    if genre in unique:
        return counts[unique==genre][0]
    return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
import numpy as np
rock_rdd=cleaned.map(lambda x: (x[0],get_trend(x[1],'rock and indie') ))
classic_rdd = cleaned.map(lambda x: (x[0],get_trend(x[1],'classic pop and rock') ))
hiphop_rdd = cleaned.map(lambda x: (x[0],get_trend(x[1],'hip hop rnb and dance hall') ))
electronica_rdd = cleaned.map(lambda x: (x[0],get_trend(x[1],'dance and electronica') ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
all_rdd=rock_rdd.join(classic_rdd,numPartitions=partition_ct).join(hiphop_rdd,numPartitions=partition_ct).join( electronica_rdd,numPartitions=partition_ct ).map(lambda x : [x[0],
                                                                                            int(x[1][0][0][0]),
                                                                                            int ( x[1][0][0][1]),
                                                                                            int( x[1][0][1]),
                                                                                             int(x[1][1])]).filter(lambda x:x[0]>1960).sortBy(lambda x:x[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:

viz3_df=all_rdd.toDF()
viz3_df.createOrReplaceTempView("viz3_tab")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
%%sql -o analysis3_df
SELECT * FROM viz3_tab 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [23]:
%%local
import plotly.express as px
analysis3_df.columns=["year","rock and indie","clasic pop and rock", "hip hop","dance and electronica"]
fig = px.line(analysis3_df, x="year", y=["rock and indie","clasic pop and rock", "hip hop","dance and electronica"])
fig.update_layout(
    height=400,
    title_text='Evolution of different genre of songs',
    yaxis_title="#Number of songs",
    xaxis_title="Year",
)
fig.show()

In [24]:
end = datetime.datetime.now()
print( end - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0:00:44.332355

In [25]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…